In [1]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geometry import Geometry
from arcgis.gis import GIS
import arcpy
import os

In [2]:
#: WARNING: if you do this twice with the same geometry, you may overwrite the other column and lose it forever. 
#: I haven't tested that yet.

def _change_geometry(df, new_geometry_column, name_for_current):
    df[name_for_current] = df['SHAPE']
    df['SHAPE'] = df[new_geometry_column]
    df.spatial.set_geometry('SHAPE')
    df.spatial.sindex(reset=True)  #: not sure how necessary this is, but for safety's sake

In [3]:
def _dissolve_duplicate_parcels(parcels_for_modeling_layer):
    # dissolve on parcel id,summarizing attributes in various ways
    parcels_dissolved = 'memory/dissolved'
    if arcpy.Exists(parcels_dissolved):
        arcpy.management.Delete(parcels_dissolved)
    arcpy.management.Dissolve(
        parcels_for_modeling_layer, parcels_dissolved, 'PARCEL_ID', [
            ['PARCEL_ID', 'COUNT'],
            ['TAXEXEMPT_TYPE', 'FIRST'],
            ['TOTAL_MKT_VALUE', 'SUM'],
            ['LAND_MKT_VALUE', 'SUM'],
            ['PARCEL_ACRES', 'SUM'],
            ['PROP_CLASS', 'FIRST'],
            ['PRIMARY_RES', 'FIRST'],
            ['HOUSE_CNT', 'MAX'],
            ['BLDG_SQFT', 'SUM'],
            ['FLOORS_CNT', 'MAX'],
            ['BUILT_YR', 'FIRST'],
            ['EFFBUILT_YR', 'FIRST'],
        ], 'MULTI_PART', 'DISSOLVE_LINES'
    )

    # rename columns
    #: moves OBJECTID to first column
    parcels_dissolved_sdf = pd.DataFrame.spatial.from_featureclass(parcels_dissolved)
    parcels_dissolved_sdf.columns = [
        'OBJECTID',
        'PARCEL_ID',
        'COUNT_PARCEL_ID',
        'TAXEXEMPT_TYPE',
        'TOTAL_MKT_VALUE',
        'LAND_MKT_VALUE',
        'PARCEL_ACRES',
        'PROP_CLASS',
        'PRIMARY_RES',
        'HOUSE_CNT',
        'BLDG_SQFT',
        'FLOORS_CNT',
        'BUILT_YR',
        'EFFBUILT_YR',
        'SHAPE',
    ]

    # remove parcels without parcel ids
    # parcels_dissolved_sdf = parcels_dissolved_sdf[parcels_dissolved_sdf['PARCEL_ID'].isnull() == False].copy()
    # parcels_dissolved_sdf = parcels_dissolved_sdf[parcels_dissolved_sdf['PARCEL_ID'].notnull()].copy()
    parcels_dissolved_sdf.dropna(subset=['PARCEL_ID'], inplace=True)

    return parcels_dissolved_sdf

# Join test

In [4]:
areas_fc = r'c:/gis/projects/udoh/udoh.gdb/SmallStatisticalAreas_2018'
tracts_fc = r'C:\Users\jdadams\AppData\Roaming\Esri\ArcGISPro\Favorites\opensgid.agrc.utah.gov.sde\opensgid.demographic.census_tracts_2020'

In [25]:
areas_df = pd.DataFrame.spatial.from_featureclass(areas_fc)
tracts_df = pd.DataFrame.spatial.from_featureclass(tracts_fc)

In [6]:
tracts_df['CENTROIDS'] = tracts_df['SHAPE'].apply(
    lambda shape: Geometry({
        'x': shape.centroid[0],
        'y': shape.centroid[1],
        'spatialReference': shape.spatial_reference
    })
)

In [18]:
m1 = GIS().map()
m1.add_layer(tracts_df)
m1.add_layer(areas_df)
m1

MapView(layout=Layout(height='400px', width='100%'))

In [7]:
_change_geometry(tracts_df, 'CENTROIDS', 'POLYS')

In [8]:
joined_df = tracts_df.spatial.join(areas_df, 'left', 'within')

In [9]:
joined_df

,xid,statefp20,countyfp20,tractce20,geoid20,name20,namelsad20,mtfcc20,funcstat20,aland20,awater20,intptlat20,intptlon20,SHAPE,CENTROIDS,POLYS,index_right,OBJECTID,smallareanumber,smallareaname
0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,"{""x"": 441169.29876317625, ""y"": 4461854.3007924...","{'x': 441169.29876317625, 'y': 4461854.3007924...","{""rings"": [[[440398.4642000003, 4462604.4892],...",59.0,63.0,44,Orem (North)
1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,"{""x"": 444725.4420491648, ""y"": 4453445.15503423...","{'x': 444725.4420491648, 'y': 4453445.15503423...","{""rings"": [[[444113.66309999954, 4453163.1712]...",64.0,68.0,48.2,Provo (East City Center)
2,3,49,049,003401,49049003401,34.01,Census Tract,G5020,S,9.085765e+06,23788.0,+40.0560227,-111.7122570,"{""x"": 439257.47857315646, ""y"": 4434122.5080396...","{'x': 439257.47857315646, 'y': 4434122.5080396...","{""rings"": [[[436447.3992999997, 4432868.571], ...",71.0,75.0,50.2,Payson
3,4,49,049,010210,49049010210,102.10,Census Tract,G5020,S,1.581048e+07,0.0,+40.4005884,-111.7306856,"{""x"": 437994.61392989865, ""y"": 4472476.4166772...","{'x': 437994.61392989865, 'y': 4472476.4166772...","{""rings"": [[[434385.5806999998, 4474166.1998],...",58.0,62.0,43,Pleasant Grove/Lindon
4,5,49,049,000505,49049000505,5.05,Census Tract,G5020,S,3.919712e+06,0.0,+40.3700595,-111.7070846,"{""x"": 439970.29811329854, ""y"": 4469071.4450746...","{'x': 439970.29811329854, 'y': 4469071.4450746...","{""rings"": [[[438303.71279999986, 4469684.0035]...",58.0,62.0,43,Pleasant Grove/Lindon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,712,49,035,111701,49035111701,1117.01,Census Tract,G5020,S,1.358256e+06,0.0,+40.6934670,-111.8826082,"{""x"": 425428.02143853245, ""y"": 4505104.4082948...","{'x': 425428.02143853245, 'y': 4505104.4082948...","{""rings"": [[[424945.5695000002, 4504747.2454],...",23.0,27.0,25,South Salt Lake
712,713,49,035,111903,49035111903,1119.03,Census Tract,G5020,S,2.072253e+06,0.0,+40.6806830,-111.8627470,"{""x"": 427092.1915410602, ""y"": 4503668.64664819...","{'x': 427092.1915410602, 'y': 4503668.64664819...","{""rings"": [[[426353.4841, 4503135.8453], [4263...",25.0,29.0,26.2,Millcreek (South)
713,714,49,035,111905,49035111905,1119.05,Census Tract,G5020,S,1.167718e+06,0.0,+40.6765749,-111.8744366,"{""x"": 426099.79680161, ""y"": 4503222.390862818,...","{'x': 426099.79680161, 'y': 4503222.390862818,...","{""rings"": [[[425548.92899999954, 4502917.7248]...",34.0,38.0,31,Murray
714,715,49,035,113309,49035113309,1133.09,Census Tract,G5020,S,1.867612e+06,0.0,+40.6881369,-111.9693980,"{""x"": 418088.48904618004, ""y"": 4504590.0128683...","{'x': 418088.48904618004, 'y': 4504590.0128683...","{""rings"": [[[417449.58469999954, 4505538.77260...",20.0,24.0,23.1,West Valley (East) V2


In [9]:
joined_df[joined_df.isna().any(axis=1)].shape[0]

1

In [10]:
joined_df[joined_df['smallareaname'].notna()]

,xid,statefp20,countyfp20,tractce20,geoid20,name20,namelsad20,mtfcc20,funcstat20,aland20,awater20,intptlat20,intptlon20,SHAPE,CENTROIDS,POLYS,index_right,OBJECTID,smallareanumber,smallareaname
0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,"{""x"": 441169.29876317625, ""y"": 4461854.3007924...","{'x': 441169.29876317625, 'y': 4461854.3007924...","{""rings"": [[[440398.4642000003, 4462604.4892],...",59.0,63.0,44,Orem (North)
1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,"{""x"": 444725.4420491648, ""y"": 4453445.15503423...","{'x': 444725.4420491648, 'y': 4453445.15503423...","{""rings"": [[[444113.66309999954, 4453163.1712]...",64.0,68.0,48.2,Provo (East City Center)
2,3,49,049,003401,49049003401,34.01,Census Tract,G5020,S,9.085765e+06,23788.0,+40.0560227,-111.7122570,"{""x"": 439257.47857315646, ""y"": 4434122.5080396...","{'x': 439257.47857315646, 'y': 4434122.5080396...","{""rings"": [[[436447.3992999997, 4432868.571], ...",71.0,75.0,50.2,Payson
3,4,49,049,010210,49049010210,102.10,Census Tract,G5020,S,1.581048e+07,0.0,+40.4005884,-111.7306856,"{""x"": 437994.61392989865, ""y"": 4472476.4166772...","{'x': 437994.61392989865, 'y': 4472476.4166772...","{""rings"": [[[434385.5806999998, 4474166.1998],...",58.0,62.0,43,Pleasant Grove/Lindon
4,5,49,049,000505,49049000505,5.05,Census Tract,G5020,S,3.919712e+06,0.0,+40.3700595,-111.7070846,"{""x"": 439970.29811329854, ""y"": 4469071.4450746...","{'x': 439970.29811329854, 'y': 4469071.4450746...","{""rings"": [[[438303.71279999986, 4469684.0035]...",58.0,62.0,43,Pleasant Grove/Lindon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,712,49,035,111701,49035111701,1117.01,Census Tract,G5020,S,1.358256e+06,0.0,+40.6934670,-111.8826082,"{""x"": 425428.02143853245, ""y"": 4505104.4082948...","{'x': 425428.02143853245, 'y': 4505104.4082948...","{""rings"": [[[424945.5695000002, 4504747.2454],...",23.0,27.0,25,South Salt Lake
712,713,49,035,111903,49035111903,1119.03,Census Tract,G5020,S,2.072253e+06,0.0,+40.6806830,-111.8627470,"{""x"": 427092.1915410602, ""y"": 4503668.64664819...","{'x': 427092.1915410602, 'y': 4503668.64664819...","{""rings"": [[[426353.4841, 4503135.8453], [4263...",25.0,29.0,26.2,Millcreek (South)
713,714,49,035,111905,49035111905,1119.05,Census Tract,G5020,S,1.167718e+06,0.0,+40.6765749,-111.8744366,"{""x"": 426099.79680161, ""y"": 4503222.390862818,...","{'x': 426099.79680161, 'y': 4503222.390862818,...","{""rings"": [[[425548.92899999954, 4502917.7248]...",34.0,38.0,31,Murray
714,715,49,035,113309,49035113309,1133.09,Census Tract,G5020,S,1.867612e+06,0.0,+40.6881369,-111.9693980,"{""x"": 418088.48904618004, ""y"": 4504590.0128683...","{'x': 418088.48904618004, 'y': 4504590.0128683...","{""rings"": [[[417449.58469999954, 4505538.77260...",20.0,24.0,23.1,West Valley (East) V2


In [10]:
joined_inner_df = tracts_df.spatial.join(areas_df, 'inner', 'within')

In [11]:
joined_inner_df

,xid,statefp20,countyfp20,tractce20,geoid20,name20,namelsad20,mtfcc20,funcstat20,aland20,awater20,intptlat20,intptlon20,SHAPE,CENTROIDS,POLYS,index_right,OBJECTID,smallareanumber,smallareaname
0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,"{""x"": 441169.29876317625, ""y"": 4461854.3007924...","{'x': 441169.29876317625, 'y': 4461854.3007924...","{'rings': [[[440398.4642000003, 4462604.4892],...",59,63,44,Orem (North)
1,13,49,049,000703,49049000703,7.03,Census Tract,G5020,S,1.831747e+06,0.0,+40.3156465,-111.6899270,"{""x"": 441379.8662237534, ""y"": 4463020.20262828...","{'x': 441379.8662237534, 'y': 4463020.20262828...","{'rings': [[[440128.60699999984, 4463430.2863]...",59,63,44,Orem (North)
2,123,49,049,000707,49049000707,7.07,Census Tract,G5020,S,1.532480e+06,0.0,+40.3258061,-111.7137593,"{""x"": 439364.00949996844, ""y"": 4464163.9476659...","{'x': 439364.00949996844, 'y': 4464163.9476659...","{'rings': [[[438670.8861999996, 4464615.9099],...",59,63,44,Orem (North)
3,124,49,049,000801,49049000801,8.01,Census Tract,G5020,S,1.833326e+06,0.0,+40.3081523,-111.7127586,"{""x"": 439433.2507907797, ""y"": 4462203.73230499...","{'x': 439433.2507907797, 'y': 4462203.73230499...","{'rings': [[[438328.6040000003, 4462625.9417],...",59,63,44,Orem (North)
4,125,49,049,000706,49049000706,7.06,Census Tract,G5020,S,2.184751e+06,0.0,+40.3177123,-111.7170691,"{""x"": 439075.5541741788, ""y"": 4463267.82426487...","{'x': 439075.5541741788, 'y': 4463267.82426487...","{'rings': [[[437849.7767000003, 4464232.577500...",59,63,44,Orem (North)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,716,49,015,976500,49015976500,9765,Census Tract,G5020,S,8.782568e+09,14165655.0,+38.9311083,-110.6644589,"{""x"": 531221.4397219317, ""y"": 4305075.81417691...","{'x': 531221.4397219317, 'y': 4305075.81417691...","{'rings': [[[473273.6588000003, 4276684.8684],...",83,87,56.2,Emery County
711,651,49,037,978200,49037978200,9782,Census Tract,G5020,S,9.296421e+09,192709561.0,+37.6174788,-109.9834213,"{""x"": 589598.9234787896, ""y"": 4162715.97376665...","{'x': 589598.9234787896, 'y': 4162715.97376665...","{'rings': [[[508811.7142000003, 4114325.200899...",86,90,57.4,San Juan County (Other)
712,652,49,037,942100,49037942100,9421,Census Tract,G5020,S,2.962764e+09,74416830.0,+37.1031945,-110.3979381,"{""x"": 553426.9994610251, ""y"": 4107382.06255905...","{'x': 553426.9994610251, 'y': 4107382.06255905...","{'rings': [[[463282.94570000004, 4095174.705],...",86,90,57.4,San Juan County (Other)
713,654,49,037,942000,49037942000,9420,Census Tract,G5020,S,2.220605e+09,8760375.0,+37.1948026,-109.3389774,"{""x"": 646718.0546056273, ""y"": 4116071.81085491...","{'x': 646718.0546056273, 'y': 4116071.81085491...","{'rings': [[[612062.9029999999, 4111103.7786],...",86,90,57.4,San Juan County (Other)


In [12]:
joined_inner_df.groupby('smallareaname')['aland20'].sum()

smallareaname
Alpine                          8.864834e+07
American Fork                   5.599178e+07
Ben Lomond                      3.395271e+08
Blanding/Monticello             5.773384e+09
Bountiful                       9.298409e+07
                                    ...     
West Jordan (West)/Copperton    2.316268e+07
West Valley (Center)            1.631328e+07
West Valley (East) V2           2.805217e+07
West Valley (West) V2           3.281206e+07
Woods Cross/West Bountiful      3.164638e+07
Name: aland20, Length: 99, dtype: float64

In [16]:
areas_inner_df = areas_df.spatial.join(tracts_df, 'inner', 'contains')
areas_inner_df.groupby('smallareaname')['aland20'].sum()

smallareaname
Alpine                          8.864834e+07
American Fork                   5.599178e+07
Ben Lomond                      3.395271e+08
Blanding/Monticello             5.773384e+09
Bountiful                       9.298409e+07
                                    ...     
West Jordan (West)/Copperton    2.316268e+07
West Valley (Center)            1.631328e+07
West Valley (East) V2           2.805217e+07
West Valley (West) V2           3.281206e+07
Woods Cross/West Bountiful      3.164638e+07
Name: aland20, Length: 99, dtype: float64

In [23]:
areas_indexed_df = areas_df.set_index(['smallareaname'])

KeyError: "None of ['smallareaname'] are in the columns"

In [24]:
areas_inner_df = areas_df.spatial.join(tracts_df, 'inner', 'contains')
areas_inner_df

KeyError: "None of ['index_left'] are in the columns"

# Davis testing

In [3]:
davis_parcels_fc = r'C:\gis\git\housing-characteristics\Parcels\2020-Davis\Inputs\Davis_County_LIR_Parcels.gdb\Parcels_Davis_LIR_UTM12'
davis_common_areas_fc = r'C:\gis\git\housing-characteristics\Parcels\2020-Davis\Inputs\Common_Areas.gdb\Common_Areas_Reviewed'


In [4]:
davis_parcels_df = pd.DataFrame.spatial.from_featureclass(davis_parcels_fc)
davis_common_areas_df = pd.DataFrame.spatial.from_featureclass(davis_common_areas_fc)

In [5]:
davis_parcels_df.shape

(119357, 30)

In [6]:
davis_common_areas_df.shape

(574, 6)

In [56]:
davis_dissolved_parcels_df = _dissolve_duplicate_parcels(davis_parcels_fc)

In [59]:
davis_parcels_df = davis_dissolved_parcels_df.copy()
davis_parcels_df['CENTROIDS'] = davis_parcels_df['SHAPE'].apply(
    lambda shape: Geometry({
        'x': shape.centroid[0],
        'y': shape.centroid[1],
        'spatialReference': shape.spatial_reference
    })
)

In [60]:
_change_geometry(davis_parcels_df, 'CENTROIDS', 'POLYS')

In [9]:
#: common areas has an empty geography! 
davis_common_areas_df[davis_common_areas_df['SHAPE'].isna()]


,OBJECTID,SUBTYPE_WFRC,Review,MERGE_SRC,TYPE_WFRC,SHAPE
291,293,condo,1,Common_Areas_Josh,multi_family,None


In [61]:
def _remove_empty_geometries(dfs, shape_field='SHAPE'):
    for df in dfs:
        df.dropna(subset=[shape_field], inplace=True)

In [62]:
_remove_empty_geometries([davis_common_areas_df, davis_parcels_df])

In [63]:
davis_inner_df = davis_parcels_df.spatial.join(davis_common_areas_df, 'inner', 'within')

In [64]:
davis_inner_df.shape

(12604, 23)

In [65]:
davis_inner_df[['PARCEL_ID', 'OBJECTID_right', 'TYPE_WFRC', 'SHAPE']].spatial.to_featureclass(r'c:\gis\projects\housinginventory\housinginventory.gdb\cenroids_df_inner_join')


C:\Users\jdadams\Miniconda3\envs\housing\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'c:\\gis\\Projects\\housinginventory\\housinginventory.gdb\\cenroids_df_inner_join'

In [66]:
davis_inner_df.head()

,OBJECTID_left,PARCEL_ID,COUNT_PARCEL_ID,TAXEXEMPT_TYPE,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,BLDG_SQFT,FLOORS_CNT,BUILT_YR,EFFBUILT_YR,SHAPE,CENTROIDS,POLYS,index_right,OBJECTID_right,SUBTYPE_WFRC,Review,MERGE_SRC,TYPE_WFRC
0,2993,011270002,1.0,NO,153000.0,25500.0,0.01,Residential,Y,1,980.0,2.0,1980.0,None,"{""x"": 423098.2202059751, ""y"": 4521292.73668173...","{'x': 423098.2202059751, 'y': 4521292.73668173...","{'rings': [[[423095.46829999983, 4521288.78460...",216,218,townhome,1,Common_Areas_Josh,multi_family
1,2994,011270003,1.0,NO,158000.0,38000.0,0.01,Residential,Y,1,980.0,2.0,1980.0,None,"{""x"": 423092.8142552632, ""y"": 4521292.23292286...","{'x': 423092.8142552632, 'y': 4521292.23292286...","{'rings': [[[423090.1489000004, 4521296.3013],...",216,218,townhome,1,Common_Areas_Josh,multi_family
2,2995,011270004,1.0,NO,158000.0,38000.0,0.01,Residential,Y,1,980.0,2.0,1980.0,None,"{""x"": 423087.41084402276, ""y"": 4521292.2796537...","{'x': 423087.41084402276, 'y': 4521292.2796537...","{'rings': [[[423090.1489000004, 4521296.3013],...",216,218,townhome,1,Common_Areas_Josh,multi_family
3,2996,011270005,1.0,NO,158000.0,38000.0,NaN,Residential,Y,1,980.0,2.0,1980.0,None,"{""x"": 423103.29551107954, ""y"": 4521275.8902960...","{'x': 423103.29551107954, 'y': 4521275.8902960...","{'rings': [[[423100.6451000003, 4521279.9691],...",216,218,townhome,1,Common_Areas_Josh,multi_family
4,2997,011270006,1.0,NO,174000.0,38000.0,0.01,Residential,Y,1,980.0,2.0,1980.0,None,"{""x"": 423097.89209049527, ""y"": 4521275.9370569...","{'x': 423097.89209049527, 'y': 4521275.9370569...","{'rings': [[[423100.5574000003, 4521271.868699...",216,218,townhome,1,Common_Areas_Josh,multi_family


In [68]:
#: TODO: figure out why dataframe centroid inner join and arcpy centroids are giving different number of output points. Maybe export parcels as polys, intersect with subsetted main parcels to see what additional parcels i'm getting?
_change_geometry(davis_inner_df, 'POLYS', 'CENTROIDS')
# _change_geometry(davis_dissolved_parcles_df, 'POLYS', 'CENTROIDS')
davis_inner_df[['PARCEL_ID', 'OBJECTID_right', 'TYPE_WFRC', 'SHAPE']].spatial.to_featureclass(r'c:\gis\projects\housinginventory\housinginventory.gdb\cenroids_df_inner_join_polys')
davis_dissolved_parcels_df.spatial.to_featureclass(r'c:\gis\projects\housinginventory\housinginventory.gdb\davis_parcels_dissolved')

C:\Users\jdadams\Miniconda3\envs\housing\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'c:\\gis\\Projects\\housinginventory\\housinginventory.gdb\\davis_parcels_dissolved'

In [17]:
arcgis_parcels = r'c:\gis\projects\housinginventory\housinginventory.gdb\Pro_centroids_select_within'
arcgis_df = pd.DataFrame.spatial.from_featureclass(arcgis_parcels)

In [18]:
#: Excluding outer join
difference = davis_inner_df.merge(arcgis_df, on='PARCEL_ID', how='left', indicator=True).query('_merge == "left_only"').drop('_merge', 1)


C:\Users\jdadams\Miniconda3\envs\housing\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [23]:
difference.rename({'SHAPE_x':'SHAPE'}, axis=1, inplace=True)
difference.spatial.set_geometry('SHAPE')

In [29]:
difference.spatial.to_featureclass(r'c:\gis\projects\housinginventory\housinginventory.gdb\difference_parcels')

In [28]:
difference.shape

(34, 68)

In [36]:
difference.columns

Index(['OBJECTID_left', 'COUNTY_NAME_x', 'COUNTY_ID_x', 'ASSESSOR_SRC_x',
       'BOUNDARY_SRC_x', 'DISCLAIMER_x', 'CURRENT_ASOF_x', 'PARCEL_ID',
       'SERIAL_NUM_x', 'PARCEL_ADD_x', 'PARCEL_CITY_x', 'TAXEXEMPT_TYPE_x',
       'TAX_DISTRICT_x', 'TOTAL_MKT_VALUE_x', 'LAND_MKT_VALUE_x',
       'PARCEL_ACRES_x', 'PROP_CLASS_x', 'PRIMARY_RES_x', 'HOUSE_CNT_x',
       'SUBDIV_NAME_x', 'BLDG_SQFT_x', 'BLDG_SQFT_INFO_x', 'FLOORS_CNT_x',
       'FLOORS_INFO_x', 'BUILT_YR_x', 'EFFBUILT_YR_x', 'CONST_MATERIAL_x',
       'x_x', 'y_x', 'SHAPE_x', 'CENTROIDS', 'POLYS', 'index_right',
       'OBJECTID_right', 'SUBTYPE_WFRC', 'Review', 'MERGE_SRC', 'TYPE_WFRC',
       'OBJECTID', 'COUNTY_NAME_y', 'COUNTY_ID_y', 'ASSESSOR_SRC_y',
       'BOUNDARY_SRC_y', 'DISCLAIMER_y', 'CURRENT_ASOF_y', 'SERIAL_NUM_y',
       'PARCEL_ADD_y', 'PARCEL_CITY_y', 'TAXEXEMPT_TYPE_y', 'TAX_DISTRICT_y',
       'TOTAL_MKT_VALUE_y', 'LAND_MKT_VALUE_y', 'PARCEL_ACRES_y',
       'PROP_CLASS_y', 'PRIMARY_RES_y', 'HOUSE_CNT_y', 

In [42]:
davis_dissolved_parcels_df = _dissolve_duplicate_parcels(davis_parcels_fc)

In [43]:
davis_dissolved_parcels_df.shape

(114047, 15)

In [45]:
dup_parcel_ids = davis_parcels_df[davis_parcels_df.duplicated(subset=['PARCEL_ID'])]

In [49]:
dup_parcel_ids[dup_parcel_ids['PARCEL_ID'].isna()].shape

(604, 32)

In [50]:
pd.set_option('display.max_columns', None)

In [55]:
davis_dissolved_parcels_df.sort_values(by=['COUNT_PARCEL_ID'], ascending=False)
# davis_dissolved_parcels_df

,OBJECTID,PARCEL_ID,COUNT_PARCEL_ID,TAXEXEMPT_TYPE,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,BLDG_SQFT,FLOORS_CNT,BUILT_YR,EFFBUILT_YR,SHAPE
56696,56697,093660015,12.0,COMMON,600.0,600.0,1.200,Vacant Land,N,None,0.0,0.0,NaN,None,"{""rings"": [[[423407.31819999963, 4548329.5931]..."
83369,83370,121260110,11.0,YES,77506.0,77506.0,2.882,Vacant Land,N,None,0.0,0.0,NaN,None,"{""rings"": [[[413167.71779999975, 4547802.1066]..."
98740,98741,130600034,11.0,NO,50380000.0,3350622.0,16.005,Commercial,N,16,705309.0,1.0,1981.0,None,"{""rings"": [[[413828.1775000002, 4556043.2356],..."
6357,6358,013490009,10.0,NO,89892350.0,30265430.0,513.140,Residential,Y,5,195810.0,1.0,2012.0,None,"{""rings"": [[[426978.04839999974, 4521322.22980..."
100862,100863,131370044,9.0,YES,885555.0,885555.0,1.890,Vacant Land,N,None,0.0,0.0,NaN,None,"{""rings"": [[[422536.15670000017, 4552418.81980..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39091,39092,072840122,1.0,COMMON,35.0,35.0,0.070,Vacant Land,N,None,0.0,0.0,NaN,None,"{""rings"": [[[422435.1802000003, 4541558.1943],..."
39090,39091,072840121,1.0,NO,345000.0,79300.0,0.050,Residential,Y,1,989.0,1.0,2014.0,None,"{""rings"": [[[422431.71389999986, 4541553.94270..."
39089,39090,072840120,1.0,NO,366000.0,79300.0,0.050,Residential,Y,1,1143.0,1.0,2014.0,None,"{""rings"": [[[422425.10259999987, 4541559.3331]..."
39088,39089,072840119,1.0,NO,364000.0,79300.0,0.050,Residential,Y,1,1143.0,1.0,2014.0,None,"{""rings"": [[[422418.4912999999, 4541564.723200..."
